###### Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c)2015 L.A. Barba, G.F. Forsyth, P.Y. Chuang. 

# Relax and hold steady

Welcome to the fifth notebook of *Relax and hold steady: elliptic problems*, the fifth module of [**"Practical Numerical Methods with Python"**](http://openedx.seas.gwu.edu/courses/GW/MAE6286/2014_fall/about).

We have a collection of multigrid tools and now it's time to put them to use.  Note that all of the functions created in the previous notebook have been dropped into a 'helper' file called `multigrid_helper.py`.  

In [ ]:
import numpy
import time
from numba import jit
from multigrid_helper import RMS, full_weighting_1d, interpolation_1d, \
                                poisson1d_GS_SingleItr, residual

Now that we have all of the tools we need for multigrid, let's go through the exact procedure for putting them to use.  

The two-grid procedure detailed below assumes (you guessed it) that we have two grids where the coarse grid has half as many points as the fine grid.  

### Two-grid correction scheme: detailed procedure

**Note:** superscripts here represent the order of ***cycle*** rather than the total number of iterations.  

1. Relax the original Laplace/Poisson equation on the fine grid with $n_f$ iterations and initial guess $p{^0_i}$.
2. Compute the residual $r_{i,f}$ on the fine grid.
3. Transfer $r_{i,f}$ to the coarse grid by restriction and obtain $r_{i,c}$.
4. Relax the residual equation on the coarse grid with $n_c$ iterations and initial guess $e_{i,c}=0$.
5. Transfer relaxed $e_{i,c}$ to the fine grid by interpolation and obtain $e_{i,f}$.
6. Calculate a new approximate solution by $p{^{0'}_i}=p{^0_i}+e_{i,f}$.
7. Relax the original Laplace/Poisson equation on the fine grid with $n_f$ iterations and $p{^{0'}_i}$ to obtain the final approximated solution of this cycle, i.e. $p{^1_i}$
8. Repeat until specified tolerance is met or maximum cycle count is reached.

For the example problem we are using a fine grid with $N_{x,f}=513$.  The initial guess $p^0_i$ is a wave constructed with a mixture of wavenumbers. 

In the first run, the number of internal iterations for steps $1$, $4$ and $7$ ($n_f$ and $n_c$) will be set equal to $1$

The criteria for the loop to stop is that the relative $L_2$ error reaches $10^{-8}$ or the number of cycles reaches $10^{10}$.

### Example 1: 1D Poisson Equation

Recall that the example problem that we used to develop the multigrid tools was a 1D Poisson problem with Dirichlet boundary conditions.  

\begin{equation}
\frac{p_{i+1}-2p_i+p_{i-1}}{\Delta x^2}=b_i ,\ \ \ \ \ 0 \le i \le N_x-1
\end{equation}

In [ ]:
def laplace1d_mixed_IC(nx, k):
    '''
    Generates initial guess for Laplace 1D eq. under a given number 
    of grid points (nx) and a mixture of wavenumbers within the domain [0,1]x[0,1]
    
    Parameters:
    ----------
    nx: int, number of grid points in x direction
    k:  1D list or numpy array, wavenumbers
    
    Returns:
    -------
    p: float, 1D array, unknowns in the Poisson eq.
    b: float, 1D array, 0th-order derivatives term in the Poisson eq.
    x: float, 1D array, linspace coordinates in x
    dx: float, grid spacing in x
    '''
    
    dx = 1.0 / (nx - 1)
    
    ##initial conditions
    p = numpy.zeros(nx)
    i = numpy.arange(0, nx)
    for ki in k:
        p += numpy.sin(i * ki * numpy.pi / (nx-1))

    p /= numpy.size(k)
    b = numpy.zeros(nx)
    
    return p, b, dx

In [ ]:
def L2_error(p, pn):
    '''
    Calculate relative L2 error between p and pn.
    '''
    return numpy.sqrt(numpy.sum((p - pn)**2)/numpy.sum(pn**2))

In [ ]:
# set up the # of grid points on the fine grid
nxf = 513

# set up the wavenumbers
k = [1, (nxf-1)/4, (nxf-1)/2, (nxf-1)*3/4, (nxf-2)]

# initialize the mixture-wavenumber IC
p, b, dxf = laplace1d_mixed_IC(nxf, k)

# set up the # of grid points and spacing on the coarse grid
nxc = (nxf + 1) / 2
dxc = dxf * 2

# initialize the memory space for residual and errors on the both grids
rf = numpy.zeros(nxf)
rc = numpy.zeros(nxc)
ef = numpy.zeros(nxf)
ec = numpy.zeros(nxc)

# set up the criteria for the loop to stop
tol = 1e-8
NitrMax = 1e10

# start to relax
err = 10.
itr = 0
t1 = time.time()
while err > tol and itr < NitrMax:
    
    # save previous approximated solution
    pn = p.copy()
    
    # relax p on fine grid
    p = poisson1d_GS_SingleItr(nxf, dxf, p, b)
    
    # calculate residual
    rf = residual(dxf, p, b, rf)
    
    # transfer residual to the coarse grid
    rc = full_weighting_1d(rf, rc)
    
    # relax the error on the coarse grid
    ec[...] = 0
    ec = poisson1d_GS_SingleItr(nxc, dxc, ec, rc)
    
    # transfer the error to the fine grid
    ef = interpolation_1d(ec, ef)
    
    # add the error back to p
    p += ef
    
    # relax p on the fine grid
    p = poisson1d_GS_SingleItr(nxf, dxf, p, b)

    # update L2 error and # of cycle
    err = L2_error(p, pn)
    itr += 1

t2 = time.time()

print("# of Grid points: {}".format(nxf))
print("Runtime (multigrid, two-grid correction): {:.4f} sec".format(t2-t1))
print("Root mean square of exact error: {:.4} ".format(RMS(p)))

Ok!  A fine grid with 513 points took around 15 seconds to finish on a 1.9 GHz processor.

Is that any good?  Time to find out!

Let's try the same problem using a single-grid relaxation.  Recall that the Poisson iteration function uses Gauss-Seidel, so the "single-grid" relaxation is the same thing as Gauss-Seidel from Lesson 3!

In [ ]:
# set up the # of grid points on the fine grid
nx = 513

# set up the wavenumbers
k = [1, (nx-1)/4, (nx-1)/2, (nx-1)*3/4, (nx-2)]

# initialize the mixture-wavenumber IC
p, b, dx = laplace1d_mixed_IC(nx, k)

# set up the criteria for the loop to stop
tol = 1e-8
NitrMax = 1e10

# start to relax
err = 10.
itr = 0
t1 = time.time()
while err > tol and itr < NitrMax:
    
    # save previous approximated solution
    pn = p.copy()
    
    # solve p on fine grid
    p = poisson1d_GS_SingleItr(nx, dx, p, b)
    
    # update L2 error and # of iteration
    err = L2_error(p, pn)
    itr += 1

t2 = time.time()

print("# of Grid points: {}".format(nxf))
print("Runtime (multigrid, two-grid correction): {:.4f} sec".format(t2-t1))
print("Root mean square of exact error: {:.4} ".format(RMS(p)))

That was *slow*.  But that's why we have multigrid!  Also, with the two-grid scheme, if we have a more accurate "guess" for the error relaxation, the method should converge faster.  How do we achieve a better guess for the error?  Simple.  Run the relaxation routine a few more times.  Try it out!  Wrap the line

```Python
ec = poisson1d_GS_SingleItr(nxc, dxc, ec, rc)
```

in a `for` loop and try running it 10 times (or more!) before interpolating back up to the fine mesh.  



In [ ]:
internal_itercount = 10

# set up the # of grid points on the fine grid
nxf = 513

# set up the wavenumbers
k = [1, (nxf-1)/4, (nxf-1)/2, (nxf-1)*3/4, (nxf-2)]

# initialize the mixture-wavenumber IC
p, b, dxf = laplace1d_mixed_IC(nxf, k)

# set up the # of grid points and spacing on the coarse grid
nxc = (nxf + 1) / 2
dxc = dxf * 2

# initialize the memory space for residual and errors on the both grids
rf = numpy.zeros(nxf)
rc = numpy.zeros(nxc)
ef = numpy.zeros(nxf)
ec = numpy.zeros(nxc)

# set up the criteria for the loop to stop
tol = 1e-8
NitrMax = 1e10

# start to relax
err = 10.
itr = 0
t1 = time.time()
while err > tol and itr < NitrMax:

    # save previous approximated solution
    pn = p.copy()

    # relax p on fine grid
    p = poisson1d_GS_SingleItr(nxf, dxf, p, b)

    # calculate residual
    rf = residual(dxf, p, b, rf)

    # transfer residual to the coarse grid
    rc = full_weighting_1d(rf, rc)

    # relax the error on the coarse grid
    ec[...] = 0
    for i in range(internal_itercount):
        ec = poisson1d_GS_SingleItr(nxc, dxc, ec, rc)

    # transfe the error to the fine grid
    ef = interpolation_1d(ec, ef)

    # add the error back to p
    p += ef

    # relax p on the fine grid
    p = poisson1d_GS_SingleItr(nxf, dxf, p, b)

    # update L2 error and # of cycle
    err = L2_error(p, pn)
    itr += 1

t2 = time.time()

print("# of Grid points: {}".format(nxf))
print("Runtime (multigrid, two-grid correction): {:.4f} sec".format(t2-t1))
print("Root mean square of exact error: {:.4} ".format(RMS(p)))

That made a pretty significant difference!  

##### Dig Deeper

Experiment with using different numbers of iterations for the inner-loop relaxations.  Change the grid-size of the fine and coarse grids and again experiment with different iteration counts.

### Example 2: 2D Laplace equation with Neumann boundary conditions

Let's revisit the example Laplace problem from Lesson 1 and apply the two-grid multigrid scheme. 

\begin{equation}
\frac{\partial ^2 p}{\partial x^2} + \frac{\partial ^2 p}{\partial y^2} = 0
\end{equation}

Recall that the boundary conditions were given as

\begin{equation}
\begin{array}{llll}
p&=&0 & \text{ at } x=0 \\
\frac{\partial p}{\partial x} &=& 0 & \text{ at } x = L \\
p &=& 0 & \text{ at }y = 0 \\
p &=& \sin \left(  \frac{\frac{3}{2}\pi x}{L} \right) & \text{ at } y = H
\end{array}
\end{equation}

where $L=1$ and $H=1$ are the sizes of the domain in the $x$ and $y$ directions, respectively.

We can use both single-grid and multigrid schemes to solve this problem and compare how they stack up.  

The analytical solution is

\begin{equation}
p(x,y) = \frac{\sinh \left( \frac{\frac{3}{2} \pi y}{L}\right)}{\sinh \left(  \frac{\frac{3}{2} \pi H}{L}\right)} \sin \left( \frac{\frac{3}{2} \pi x}{L} \right)
\end{equation}

In [ ]:
def p_analytical(x, y):
    '''
    Analytical solution of the model problem: 
    2D Laplace equation with Neumann BC.
    '''
    X, Y = numpy.meshgrid(x,y)
    
    p_an = numpy.sinh(1.5 * numpy.pi * Y / x[-1]) / \
           (numpy.sinh(1.5 * numpy.pi * y[-1] / x[-1])) * \
           numpy.sin(1.5 * numpy.pi * X / x[-1])
    
    return p_an

### Residual

In the previous lesson we calculated the residual in 1D using a 2nd-order central difference equation

\begin{equation}
r^k_i = b_i - \frac{p{^k_{i+1}}-2p{^k_i}+p{^k_{i-1}}}{\Delta x^2}
\end{equation}

We can extend the same residual formulation to 2D to get

\begin{equation}
r^k_i = b_i - \frac{p^{k}_{i,j-1} + p^k_{i,j+1} + p^{k}_{i-1,j} + p^k_{i+1,j}  - 4p^{k+1}_{i,j}}{\Delta x^2}
\end{equation}

For a Laplace problem, $b_i$ is always going to be zero, but we can include it here and then use the resulting code for both Laplace and Poisson problems.  

### Neumann Boundaries

The boundaries in the Poisson example were all Dirichlet boundaries, but now we have a Neumann boundary.  Not a problem.  The residual along the Dirichlet boundaries will remain zero and we calculate the residual along the Neumann boundary using the 2nd-order approximation introduced in Lesson 1.  

In [ ]:
def residual2d_neumann(dx, pn, b, r):
    '''
    Calculate the residual for the 2D Poisson equation with Neumann BC.
    Only applicable when nx=ny and dx=dy.
    
    Parameters:
    ----------
    pn: 2D array, approximated solution at a certain iteration n
    b:  2D array, the b(x) in the Poisson eq.
    
    Return:
    ----------
    The residual r
    '''
    
    # Again, we assume the boundary values of the array pn have already been 0
    # r[0, :] = 0
    # r[:, 0] = 0
    # r[nx-1, :] = 0
    
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - (pn[1:-1, :-2] + pn[1:-1, 2:] + \
                    pn[:-2, 1:-1] + pn[2:, 1:-1] - 4 * pn[1:-1, 1:-1]) / dx**2
    r[1:-1, -1] = b[1:-1, -1] - (2 * pn[1:-1, -2] + pn[:-2, -1] +\
                    pn[2:, -1] - 4 * pn[1:-1, -1]) / dx**2
        
    return r

### Restriction and interpolation in 2D

In 2D, the full weighting restriction operator is given as

\begin{align}
v_{(i,j),c} &=\frac{1}{16} \left[v_{(2i-1,2j-1),f} +v_{(2i-1,2j+1),f}\right.\notag \\ 
&\ +v_{(2i+1,2j-1),f}  +v_{(2i+1,2j+1),f}\notag \\
&\ + 2\left(v_{(2i,2j-1),f}  +v_{(2i,2j+1),f}\right.\notag \\
&\ \left.+v_{(2i-1,2j),f}  +v_{(2i+1,2j),f} \right) + \left.4v_{(2i,2j),f} \right]
\end{align}

for $1 \le i,j \le N_{x,c}-2$.

That can be a bit hard to parse, but consider the figure below where a weighted average of 9 points in the fine grid is used to calculate the value of the corresponding coarse grid point.

<img src="./figures/2d_full_weighting.svg" width=600>

#### Figure 1. Full weighting restriction in 2D

<img src="./figures/2d_full_weighting_detail.svg" width=600>

#### Figure 2. Full weighting restriction point weights

Shown in Figure $(2)$ is the weighting of the individual points in the 9-point stencil.

In [ ]:
def full_weighting_2d(vF, vC):
    '''
    2D full weighting restriction. Only applicable when nx=ny and dx=dy.
    
    Parameters
    ----------
    vF: nF x nF array, the array on the fine grid
    vC: nC x nC array, the array on the coarse grid
    
    Return
    ----------
    vC: nC x nC array, the array on the coarse grid
    '''
    vC[0, :] = vF[0, ::2]
    vC[-1, :] = vF[-1, ::2]
    vC[:, 0] = vF[::2, 0]
    vC[:, -1] = vF[::2, -1]
    vC[1:-1, 1:-1] = (vF[1:-3:2, 1:-3:2] + vF[1:-3:2, 3:-1:2] +\
                        vF[3:-1:2, 1:-3:2] + vF[3:-1:2, 3:-1:2] +\
                        2. * (vF[2:-2:2, 1:-3:2] + vF[2:-2:2, 3:-1:2] +\
                        vF[1:-3:2, 2:-2:2] + vF[3:-1:2, 2:-2:2]) +\
                        4. * vF[2:-2:2, 2:-2:2]) / 16.0
    
    return vC

In the current example we continue to inject boundary values between the fine and coarse grids.  This will return the correct answer but by "ignoring" the Neumann boundary condition the multigrid algorithm will have to cycle many more times to reduce the error sufficiently.  

To improve this, we can implement a more appropriate weighted calculation along the Neumann boundary.  Recall the implementation of the 2nd-order Neumann boundary condition from [Lesson 1](./01_2D_Laplace.ipynb).  If we extend the domain to include "ghost points" along the Neumann boundary we can then substitute values for the ghost points using the 2nd-order central difference approximation.

<img src="./figures/2d_full_weighting_neumann.svg" width=600>

#### Figure 3. Neumann Boundary Interpolation

In [ ]:
def full_weighting_2d_neumann(vF, vC):
    '''
    2D full weighting restriction. Only applicable when nx=ny and dx=dy.
    
    Parameters
    ----------
    vF: nF x nF array, the array on the fine grid
    vC: nC x nC array, the array on the coarse grid
    
    Return
    ----------
    vC: nC x nC array, the array on the coarse grid
    '''
    vC[0, :] = vF[0, ::2]
    vC[-1, :] = vF[-1, ::2]
    vC[:, 0] = vF[::2, 0]
    
    vC[1:-1, -1] = (4*vF[2:-2:2, -1] + 4*vF[2:-2:2,-2] +\
                    2*(vF[1:-3:2,-1] + vF[1:-3:2,-2] +\
                       vF[3:-1:2,-1] + vF[3:-1:2,-2]))/16.
    
    vC[1:-1, 1:-1] = (vF[1:-3:2, 1:-3:2] + vF[1:-3:2, 3:-1:2] +\
                        vF[3:-1:2, 1:-3:2] + vF[3:-1:2, 3:-1:2] +\
                        2. * (vF[2:-2:2, 1:-3:2] + vF[2:-2:2, 3:-1:2] +\
                        vF[1:-3:2, 2:-2:2] + vF[3:-1:2, 2:-2:2]) +\
                        4. * vF[2:-2:2, 2:-2:2]) / 16.0
    
    return vC

The **interpolation in 2D** is

\begin{equation}
\begin{array}{ll}
v_{(2i,2j),f}=v_{(i,j),c}, & 0 \le i,j \le N_{x,c}-1\\
v_{(2i+1,2j),f}=\frac{1}{2}\left(v_{(i,j),c}+v_{(i+1,j),c}\right),\ \ \ \ \ \ \ \ \ \ \ \ 0 \le i \le N_{x,c}-2, & 0 \le j \le N_{x,c}-1\\
v_{(2i,2j+1),f}=\frac{1}{2}\left(v_{(i,j),c}+v_{(i,j+1),c}\right),\ \ \ \ \ \ \ \ \ \ \ \ 0 \le i \le N_{x,c}-1, & 0 \le j \le N_{x,c}-2\\
v_{(2i+1,2j+1),f}=\frac{1}{4}\left(v_{(i,j),c}+v_{(i+1,j),c}+v_{(i,j+1),c}+v_{(i+1,j+1),c}\right), & 0 \le i,j \le N_{x,c}-2
\end{array}
\nonumber
\end{equation}

<img src="./figures/2d_interpolation.svg" width=500>

#### Figure 4. 2D Interpolation Weights

In [ ]:
def interpolation_2d(vC, vF):
    '''
    2D interpolation. Only applicable when nx=ny and dx=dy.
    
    Parameters
    ----------
    vC: 2D array, the array on the coarse grid
    vF: 2D array, the array on the fine grid
    
    Return
    ----------
    vF: 2D array, the array on the fine grid
    '''
    vF[::2, ::2] = vC[:, :]
    vF[1:-1:2, ::2] = 0.5 * (vC[:-1, :] + vC[1:, :])
    vF[::2, 1:-1:2] = 0.5 * (vC[:, :-1] + vC[:, 1:])
    vF[1:-1:2, 1:-1:2] = 0.25 * (vC[:-1, :-1] +\
                                 vC[1:, :-1] + vC[:-1, 1:] + vC[1:, 1:])
    
    return vF

### Gauss-Seidel method for 2D Poisson equation

We will again use Gauss-Seidel as the relaxation function. Don't forget the 2nd-order approximation of the Neumann boundary condition!

In [ ]:
@jit(nopython=True)
def poisson2d_GS_neumann_SingleItr(nx, dx, p, b):
    '''
    Solves the 2D Poisson equation with one Neumann boundary. 
    Gauss-Seidel method is used. Only applicable when nx=ny and dx=dy.
    
    Parameters:
    ----------
    nx: integer, the leading size of p
    dx: float, the spacing between two grid points
    p: nx by nx array, unknowns in the 2D Poisson eq.
    b: nx by nx array, 0th-order derivative term in the Poisson eq.
        
    Returns:
    -------
    p: nx by nx array, approximated soln. of unknowns in the 2D Poisson eq.
    '''
        
    nx, ny = p.shape
    for j in range(1,ny-1):
        for i in range(1,nx-1):
            p[j,i] = .25 * (p[j,i-1] + p[j,i+1] + p[j-1,i] + p[j+1,i] -\
                            dx**2 * b[j,i])
        p[j,-1] = .25 * (2 * p[j,-2] + p[j+1,-1] + p[j-1,-1] - \
                        dx**2 * b[j,-1])
        
    return p

### Initialization

Before we start to calculate, we will need a function to initialize the calculation.

In [ ]:
def laplace2d_IC_neumann(nx):
    '''
    Initialize the 2D Laplace problem with one Neumann boundary 
    condition. Only applicable when nx=ny and dx=dy.
    
    Parameters:
    ----------
    nx: integer, the number of grid points on the x direction.
    
    Return:
    ----------
    p: nx by nx array, unknowns in the 2D Poisson eq.
    b: nx by nx array, 0th-order derivative term in the Poisson eq.
    x: nx by 1 array, x coordinates of grid points
    y: nx by 1 array, y coordinates of grid points
    dx, dy: spacing between points
    '''
    dx = 1.0 / (nx - 1)
    dy = dx
    
    ##plotting aids
    x = numpy.linspace(0, 1, nx)
    y = x.copy()

    ##initial conditions
    p = numpy.zeros((nx, nx))

    ##Dirichlet boundary conditions
    p[-1,:] = numpy.sin(1.5 * numpy.pi * x / x[-1])
    
    b = numpy.zeros((nx, nx))
    
    return p, b, x, y, dx, dy

### Solve the 2D Poisson model problem now!

The first one here is the single-grid calculation. We use `257` grid points in the example.

In [ ]:
# set up the # of grid points, nx = ny
nx = 257

# initialize the problem
p, b, x, y, dx, dy = laplace2d_IC_neumann(nx)

# obtain the exact solution
p_exact = p_analytical(x, y)

# set up the criteria for the iteration to stop
tol = 1e-8
NitrMax = 1e10

# start the calculation
err = 10.
itr = 0
t1 = time.time()
while err > tol and itr < NitrMax:
    
    # save the old solution
    pn = p.copy()
    
    # relax p
    p = poisson2d_GS_neumann_SingleItr(nx, dx, p, b)
    
    # update the L2 error and the # of iteration
    err = L2_error(p, pn)
    itr += 1
    
t2 = time.time()

In [ ]:
print("# of Grid points: {}".format(nx*nx))
print("Runtime (single grid): {:.4f} sec".format(t2-t1))
print("Root mean square of exact error: {:.4} ".format(RMS(p - p_exact)))

Now let's see whether the multigrid scheme can accelerate the convergence. Don't forget to initialize the residual array with `numpy.zeros`, since we assume that the Dirichlet boundary values of the residual array are always `0`.

In [ ]:
internal_itercount = 10

# set up the # of grid points, nx = ny
nxf = 257

# initialize the problem
p, b, x, y, dxf, dyf = laplace2d_IC_neumann(nxf)

# obtain the exact solution
p_exact = p_analytical(x, y)

# set up the # of grid points and spacing on the coarse grid
nxc = (nxf + 1) / 2
dxc = dxf * 2

# initialize the memory space for residual and errors on the both grids
rf = numpy.zeros((nxf, nxf))
rc = numpy.zeros((nxc, nxc))
ef = numpy.zeros((nxf, nxf))
ec = numpy.zeros((nxc, nxc))

# set up the criteria for the iteration to stop
tol = 1e-8
NitrMax = 1e10

# start the calculation
err = 10.
itr = 0
t1 = time.time()
while err > tol and itr < NitrMax:
    
    # save previous approximated solution
    pn = p.copy()
    
    # relax p on fine grid
    p = poisson2d_GS_neumann_SingleItr(nxf, dxf, p, b)
    
    # calculate residual
    rf = residual2d_neumann(dxf, p, b, rf)
    
    # transfer residual to the coarse grid
    rC = full_weighting_2d_neumann(rf, rc)
    
    # relax the error on the coarse grid
    ec[...] = 0
    for i in range(internal_itercount):
        ec = poisson2d_GS_neumann_SingleItr(nxc, dxc, ec, rc)
    
    # transfe the error to the fine grid
    ef = interpolation_2d(ec, ef)
    
    # add the error back to p
    p += ef
    
    # relax p on the fine grid
    p = poisson2d_GS_neumann_SingleItr(nxf, dxf, p, b)

    # update L2 error and # of cycle
    err = L2_error(p, pn)
    itr += 1
    
t2 = time.time()

In [ ]:
print("# of Grid points: {}".format(nx*nx))
print("Runtime (multigrid, two-grid correction): {:.4f} sec".format(t2-t1))
print("Root mean square of exact error: {:.4} ".format(RMS(p - p_exact)))

Nice! We've accelerated the solution of the Laplace problem by a factor of 4 compared to the single grid method!

##### Challenge

*Try to develop your own multigrid code to solve the 2D Poisson model problem we used in the Lesson 2.*

##### Dig Deeper

The two-grid method is the simplest multigrid method.  It uses only two grids and, as a consequence, does not show off the full computational power of multigrid.

Case in point: you can go back to [Lesson 3](./Iterate.This.ipynb) and compare the performance between Optimized-SOR and the code above.  (Spoiler alert: SOR is faster).  

However, multigrid becomes more efficient as more grids are added.  As you will see in the next module on Performance Computing, full multigrid leaves everything else in the dust.  

## Reference

1. David Ketcheson, Aron Ahmadia, [Aliasing](http://nbviewer.ipython.org/github/ketch/teaching-numerics-with-notebooks/blob/master/Aliasing.ipynb)  from [*Teaching Numerics with Notebooks*](https://github.com/ketch/teaching-numerics-with-notebooks), 2014
2. William L. Griggs, Van Emden Henson, and Steve F. McCormick, [*A Multigrid Tutorial*](https://play.google.com/store/books/details/William_L_Briggs_A_Multigrid_Tutorial?id=ahklDynzz8cC), SIAM, Philadephia, 2000 
3. U. Trottenberg, C. W. Oosterlee, Anton Schüller, [*Multigrid*](https://play.google.com/store/books/details/Ulrich_Trottenberg_Multigrid?id=9ysyNPZoR24C), Academic press, 2000

4. Jonathan Shewchuk, [*An Introduction to the Conjugate Gradient Method without the Agonizing Pain*](http://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf), 1994

In [ ]:
from IPython.core.display import HTML
css_file = '../../styles/numericalmoocstyle.css'
HTML(open(css_file, "r").read())